# 5. Metaprogramowanie

# 5.1 Dekoratory

In [2]:
def generate_adder(a):
    def add(b):
        return a + b
    return add


add_10 = generate_adder(10)
add_5 = generate_adder(5)
print(add_10(100))
print(add_5(100))

110
105


In [6]:
def shouter(func):  # func == hellome
    def wrapper():  # hellome_decorated == wrapper
        print('before', func.__name__)
        func()
        print('after', func.__name__)
    return wrapper

def hellome():
    print("hellome")
    
hellome_decorated = shouter(hellome)

In [7]:
hellome()

hellome


In [8]:
hellome_decorated()

before hellome
hellome
after hellome


In [9]:
def shouter(func):  # func == hellome
    def wrapper():  # hellome_decorated == wrapper
        print('before', func.__name__)
        result = func()
        print('after', func.__name__)
        return result
    return wrapper

def hellome():
    print("hellome")
    return 42
    
hellome_decorated = shouter(hellome)

result = hellome_decorated()
print('result', result)

before hellome
hellome
after hellome
result 42


In [15]:
def shouter(func):  # func == hellome
    def wrapper():  # hellome_decorated == wrapper
        print('before', func.__name__)
        result = func()
        print('after', func.__name__)
        return result
    return wrapper

@shouter
def hellome():
    print("hellome")
    return 42

# hellome = shouter(hellome)

result = hellome()
print('result', result)

before hellome
hellome
after hellome
result 42


**Ćwiczenie.**
Zmień implementację dekoratora `@shouter` tak, aby działał także z funkcjami przyjmującymi argumenty.

In [24]:
def shouter(func):  # func == hellome
    def wrapper(*args, **kwargs):  # hellome == wrapper
        print('before', func.__name__)
        result = func(*args, **kwargs)
        print('after', func.__name__)
        return result
    return wrapper

@shouter
def hellome(a, b):
    print("hellome")
    return a+b

result = hellome(20, b=22)
print('result', result)  # ==> 'result', 42

before hellome
hellome
after hellome
result 42


In [18]:
%doctest_mode

Exception reporting mode: Plain
Doctest mode is: ON


In [33]:
from functools import wraps

def shouter(func):  # func == hellome
    @wraps(func)
    def wrapper(*args, **kwargs):  # hellome == wrapper
        print('before', func.__name__)
        result = func(*args, **kwargs)
        print('after', func.__name__)
        return result
    return wrapper

@shouter
def hellome(a, b):
    '''jreha ajfkekwaf.'''
    print("hellome")
    return a+b

result = hellome(20, b=22)
print('result', result)  # ==> 'result', 42
print(hellome.__doc__)

before hellome
hellome
after hellome
result 42
jreha ajfkekwaf.


In [36]:
from functools import wraps

class shouter:
    def __init__(?):
        ?
        
    def __call__(?):
        ?

@shouter
def hellome(a, b):
    '''jreha ajfkekwaf.'''
    print("hellome")
    return a+b

result = hellome(20, b=22)
print('result', result)  # ==> 'result', 42

SyntaxError: invalid syntax (<ipython-input-36-69e92b491573>, line 4)